In [1]:
#Load packages
import pandas as pd
import numpy as np
import scipy.io as sio
import random
import scipy
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.manifold import TSNE
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [2]:
#Load data
train = sio.loadmat('train.mat')
validation = sio.loadmat('validation.mat')
test = sio.loadmat('test_wolabels.mat')

Label encoding

In [3]:
#This function is used to encode labels since labels are categorical.
def encode_labels(labels):
    le = LabelEncoder()
    le.fit(labels)
    encoded_labels = le.transform(labels)
    
    return encoded_labels, le

def decode_labels(encoded_predict_labels, le):
    test_predictions = le.inverse_transform(encoded_predict_labels)
    
    return test_predictions

In [4]:
train_classid = np.squeeze(train['classid'])
train_class_labels = []
for item in train_classid:
    train_class_labels.append(item[0])
train_features = train['features']
train_imid = train['imid']
train_sampleid = train['sampleid']
print(train_features.shape)
train_unique_labels = sorted(np.unique(train_class_labels))
train_unique_labels_count = len(train_unique_labels)
print(train_unique_labels_count)

(7849, 2048)
1013


In [5]:
validation_classid = np.squeeze(validation['classid'])
validation_class_labels = []
for item in validation_classid:
    validation_class_labels.append(item[0])
validation_features = validation['features']
validation_imid = validation['imid']
validation_sampleid = validation['sampleid']
print(validation_features.shape)
validation_unique_labels = sorted(np.unique(validation_class_labels))
validation_unique_labels_count = len(validation_unique_labels)
print(validation_unique_labels_count)

(1379, 2048)
1013


In [6]:
#encoded train labels
train_labels, le = encode_labels(train_class_labels)
print(len(train_labels))

#encoded validation labels
validation_labels = le.transform(validation_class_labels)
print(len(validation_labels))

7849
1379


combine train and validation data 

In [7]:
features = np.vstack((train_features, validation_features))
print(features.shape)

labels = np.array(list(train_labels) + list(validation_labels))
print(len(labels))

(9228, 2048)
9228


In [8]:
#0-1 normalization
from sklearn.preprocessing import MinMaxScaler

scalar = MinMaxScaler()
features_norm = scalar.fit_transform(features)

Logistic Regression with Cross-validation Design of Experiment

In [11]:
N_components = [250, 350, 500, 750]
cost_param = [0.1, 1, 10, 100]

for n_components in N_components:
    pca = PCA(n_components=n_components)
    features_red = pca.fit_transform(features_norm)
    print("No. of principal components =", n_components)
    for c in cost_param:
        clf = LogisticRegression(C = c, class_weight = 'balanced', random_state=0)
        scores = cross_val_score(clf, features_red, labels, cv=6)
        print("Cost parameter: ", c)
        print("Mean class accuracy scores:", scores)
        print("%0.2f accuracy with a standard deviation of %0.2f" %(scores.mean(), scores.std()))

No. of principal components = 250


/home/ntalukde/anaconda3/envs/NephNet3D/lib/python3.7/site-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=6.
  UserWarning,


Cost parameter:  0.1
Mean class accuracy scores: [0.62743823 0.68530559 0.71131339 0.71716515 0.65409623 0.67685306]
0.68 accuracy with a standard deviation of 0.03


/home/ntalukde/anaconda3/envs/NephNet3D/lib/python3.7/site-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=6.
  UserWarning,


Cost parameter:  1
Mean class accuracy scores: [0.70221066 0.78608583 0.78738622 0.79778934 0.73862159 0.76592978]
0.76 accuracy with a standard deviation of 0.03


...

/home/ntalukde/anaconda3/envs/NephNet3D/lib/python3.7/site-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=6.
  UserWarning,
/home/ntalukde/anaconda3/envs/NephNet3D/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/home/ntalukde/anaconda3/envs/NephNet3D/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the numb

Cost parameter:  10
Mean class accuracy scores: [0.736671   0.80624187 0.8153446  0.8159948  0.77048114 0.79453836]
0.79 accuracy with a standard deviation of 0.03


/home/ntalukde/anaconda3/envs/NephNet3D/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/home/ntalukde/anaconda3/envs/NephNet3D/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn

Cost parameter:  100
Mean class accuracy scores: [0.74642393 0.80819246 0.82184655 0.82054616 0.76853056 0.79648895]
0.79 accuracy with a standard deviation of 0.03


/home/ntalukde/anaconda3/envs/NephNet3D/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [12]:
N_components = [600, 700, 800, 900, 1000]
cost_param = [5, 10, 50, 100, 200, 500]

for n_components in N_components:
    pca = PCA(n_components=n_components)
    features_red = pca.fit_transform(features_norm)
    print("No. of principal components =", n_components)
    for c in cost_param:
        clf = LogisticRegression(C = c, class_weight = 'balanced', random_state=0)
        scores = cross_val_score(clf, features_red, labels, cv=6)
        print("Cost parameter: ", c)
        print("Mean class accuracy scores:", scores)
        print("%0.2f accuracy with a standard deviation of %0.2f" %(scores.mean(), scores.std()))

No. of principal components = 600


/home/ntalukde/anaconda3/envs/NephNet3D/lib/python3.7/site-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=6.
  UserWarning,
/home/ntalukde/anaconda3/envs/NephNet3D/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/home/ntalukde/anaconda3/envs/NephNet3D/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the numb

Cost parameter:  5
Mean class accuracy scores: [0.73146944 0.8023407  0.81144343 0.81339402 0.76788036 0.789987  ]
0.79 accuracy with a standard deviation of 0.03


/home/ntalukde/anaconda3/envs/NephNet3D/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/home/ntalukde/anaconda3/envs/NephNet3D/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn

Cost parameter:  10
Mean class accuracy scores: [0.73407022 0.8029909  0.8153446  0.81469441 0.76853056 0.79583875]
0.79 accuracy with a standard deviation of 0.03


...

/home/ntalukde/anaconda3/envs/NephNet3D/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/home/ntalukde/anaconda3/envs/NephNet3D/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn

Cost parameter:  200
Mean class accuracy scores: [0.74642393 0.81209363 0.82444733 0.82444733 0.77373212 0.79843953]
0.80 accuracy with a standard deviation of 0.03


/home/ntalukde/anaconda3/envs/NephNet3D/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/home/ntalukde/anaconda3/envs/NephNet3D/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn

Cost parameter:  500
Mean class accuracy scores: [0.74122237 0.81404421 0.82249675 0.81989597 0.76983095 0.79843953]
0.79 accuracy with a standard deviation of 0.03


/home/ntalukde/anaconda3/envs/NephNet3D/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [9]:
N_components = [1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2048]
cost_param = [5, 10, 50, 100, 200, 500]

for n_components in N_components:
    pca = PCA(n_components=n_components)
    features_red = pca.fit_transform(features_norm)
    print("No. of principal components =", n_components)
    for c in cost_param:
        clf = LogisticRegression(C = c, class_weight = 'balanced', random_state=0)
        scores = cross_val_score(clf, features_red, labels, cv=6)
        print("Cost parameter: ", c)
        print("Mean class accuracy scores:", scores)
        print("%0.2f accuracy with a standard deviation of %0.2f" %(scores.mean(), scores.std()))

No. of principal components = 1000


/home/ntalukde/anaconda3/envs/NephNet3D/lib/python3.7/site-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=6.
  UserWarning,
/home/ntalukde/anaconda3/envs/NephNet3D/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/home/ntalukde/anaconda3/envs/NephNet3D/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the numb

Cost parameter:  5
Mean class accuracy scores: [0.736671   0.80624187 0.81469441 0.8153446  0.77243173 0.79453836]
0.79 accuracy with a standard deviation of 0.03


/home/ntalukde/anaconda3/envs/NephNet3D/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/home/ntalukde/anaconda3/envs/NephNet3D/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn

Cost parameter:  10
Mean class accuracy scores: [0.73862159 0.80819246 0.8153446  0.81794538 0.77113134 0.79648895]
0.79 accuracy with a standard deviation of 0.03


...

/home/ntalukde/anaconda3/envs/NephNet3D/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/home/ntalukde/anaconda3/envs/NephNet3D/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn

Cost parameter:  200
Mean class accuracy scores: [0.75227568 0.81404421 0.82704811 0.82249675 0.7763329  0.80169051]
0.80 accuracy with a standard deviation of 0.03


/home/ntalukde/anaconda3/envs/NephNet3D/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/home/ntalukde/anaconda3/envs/NephNet3D/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn

Cost parameter:  500
Mean class accuracy scores: [0.74707412 0.81274382 0.82704811 0.81989597 0.77308192 0.80169051]
0.80 accuracy with a standard deviation of 0.03


/home/ntalukde/anaconda3/envs/NephNet3D/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
